In [ ]:
````xml
<VSCode.Cell language="markdown">
# Microsoft Planetary Computer Data Preparation for FuseTS

This notebook extracts Sentinel-1 and Sentinel-2 data from **Microsoft Planetary Computer (MPC)** and prepares it for FuseTS MOGPR processing.

## Why Microsoft Planetary Computer?

✅ **Advantages over Google Earth Engine:**
- No authentication hassles (free public access)
- Direct access to Analysis Ready Data (ARD)
- STAC API - industry standard for satellite data
- Easy integration with Python ecosystem (xarray, rioxarray, dask)
- Cloud-optimized GeoTIFFs (COGs)
- No export/download waiting - process directly in Python

❌ **Considerations:**
- Requires more local processing/memory than GEE
- Need to handle data mosaicking yourself
- Download bandwidth may be limiting factor

## Temporal Strategy
- **Date range**: November 2023 - November 2025 (2 years)
- **Temporal resolution**: 12-day composites (dekadal periods)
- **Total periods**: ~62 periods
- **Coverage**: Full Indonesian agricultural calendar (6 growing seasons)

## Study Area
- **Location**: Klambu-Glapan paddy fields, Demak, Central Java
- **Source**: Shapefile (data/klambu-glapan.shp)
- **Buffer**: 500m around paddy boundaries

## Output Format
Data will be in FuseTS-compatible xarray format:
- S1: `VV`, `VH` bands
- S2: `S2ndvi` band
- Dimensions: `(t, y, x)` with proper CRS
</VSCode.Cell>
<VSCode.Cell language="markdown">
## 1. Setup and Install Dependencies
</VSCode.Cell>
<VSCode.Cell language="python">
# Install required packages
# Uncomment the following line if packages are not installed
# !pip install planetary-computer pystac-client rioxarray xarray numpy pandas matplotlib geopandas shapely dask

import warnings
warnings.filterwarnings('ignore')

import planetary_computer
import pystac_client
import rioxarray
import xarray as xr
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import mapping, box
from shapely.ops import unary_union
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import os
from pathlib import Path

print("✅ All packages imported successfully!")
print(f"\n📦 Package versions:")
print(f"   planetary-computer: {planetary_computer.__version__}")
print(f"   pystac-client: {pystac_client.__version__}")
print(f"   xarray: {xr.__version__}")
print(f"   rioxarray: {rioxarray.__version__}")
print(f"   geopandas: {gpd.__version__}")
</VSCode.Cell>
<VSCode.Cell language="markdown">
## 2. Load Study Area from Shapefile
</VSCode.Cell>
<VSCode.Cell language="python">
# ============================================================================
# LOAD PADDY SHAPEFILE AND DEFINE STUDY AREA
# ============================================================================

print("="*70)
print("📍 LOADING STUDY AREA FROM SHAPEFILE")
print("="*70)

# Load the paddy shapefile
shapefile_path = 'data/klambu-glapan.shp'

print(f"\n🎯 Loading: {shapefile_path}")

try:
    # Read the shapefile
    paddy_gdf = gpd.read_file(shapefile_path)
    
    print(f"\n✅ Shapefile loaded successfully!")
    print(f"   Number of features: {len(paddy_gdf)}")
    print(f"   CRS: {paddy_gdf.crs}")
    print(f"   Columns: {paddy_gdf.columns.tolist()}")
    
    # Get original bounds
    orig_west, orig_south, orig_east, orig_north = paddy_gdf.total_bounds
    print(f"\n   Original Bounds (WGS84):")
    print(f"     West:  {orig_west:.6f}°")
    print(f"     South: {orig_south:.6f}°")
    print(f"     East:  {orig_east:.6f}°")
    print(f"     North: {orig_north:.6f}°")
    
    # Convert to UTM Zone 49S for accurate area calculation and buffering
    print(f"\n   Converting to UTM Zone 49S (EPSG:32749)...")
    paddy_utm = paddy_gdf.to_crs("EPSG:32749")
    
    # Calculate area in UTM
    total_area_m2 = paddy_utm.area.sum()
    total_area_km2 = total_area_m2 / 1e6
    print(f"   Total paddy area: {total_area_km2:.2f} km²")
    
    # Apply buffer in UTM (meters)
    BUFFER_DISTANCE_M = 500
    print(f"\n   Applying {BUFFER_DISTANCE_M}m buffer...")
    
    paddy_buffered_utm = paddy_utm.copy()
    paddy_buffered_utm['geometry'] = paddy_utm.buffer(BUFFER_DISTANCE_M)
    
    # Merge all polygons
    merged_geometry_utm = unary_union(paddy_buffered_utm.geometry)
    buffered_area_km2 = merged_geometry_utm.area / 1e6
    
    print(f"   Buffered area: {buffered_area_km2:.2f} km²")
    
    # Convert back to WGS84 for MPC queries
    buffered_gdf_utm = gpd.GeoDataFrame(
        geometry=[merged_geometry_utm],
        crs="EPSG:32749"
    )
    buffered_gdf_wgs84 = buffered_gdf_utm.to_crs("EPSG:4326")
    
    # Get final bounds in WGS84
    west, south, east, north = buffered_gdf_wgs84.total_bounds
    
    print(f"\n   Final Bounds (WGS84 for MPC):")
    print(f"     West:  {west:.6f}°")
    print(f"     South: {south:.6f}°")
    print(f"     East:  {east:.6f}°")
    print(f"     North: {north:.6f}°")
    
    # Create bounding box for queries
    bbox = [west, south, east, north]
    
    # Create geometry for spatial queries
    study_area_geom = buffered_gdf_wgs84.geometry.iloc[0]
    
    print(f"\n✅ Study area prepared!")
    print(f"   Location: Klambu-Glapan, Demak, Central Java")
    print(f"   Features: {len(paddy_gdf)} paddy fields")
    print(f"   Area: {buffered_area_km2:.2f} km²")
    
    # Visualize
    fig, axes = plt.subplots(1, 2, figsize=(16, 8))
    
    # Original
    paddy_utm.plot(ax=axes[0], facecolor='lightgreen', edgecolor='darkgreen', 
                   linewidth=0.5, alpha=0.7)
    axes[0].set_title(f'Original Paddy Fields\n{total_area_km2:.2f} km²', 
                      fontsize=12, fontweight='bold')
    axes[0].set_xlabel('Easting (m UTM 49S)')
    axes[0].set_ylabel('Northing (m UTM 49S)')
    axes[0].grid(True, alpha=0.3)
    
    # Buffered
    buffered_gdf_utm.plot(ax=axes[1], facecolor='yellow', edgecolor='orange',
                          linewidth=2, alpha=0.5, label=f'{BUFFER_DISTANCE_M}m buffer')
    paddy_utm.plot(ax=axes[1], facecolor='lightgreen', edgecolor='darkgreen',
                   linewidth=0.5, alpha=0.7, label='Paddy fields')
    axes[1].set_title(f'Study Area with Buffer\n{buffered_area_km2:.2f} km²',
                      fontsize=12, fontweight='bold')
    axes[1].set_xlabel('Easting (m UTM 49S)')
    axes[1].set_ylabel('Northing (m UTM 49S)')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('mpc_study_area.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"\n   📊 Visualization saved: mpc_study_area.png")
    
except Exception as e:
    print(f"\n❌ Error loading shapefile: {e}")
    import traceback
    traceback.print_exc()
    raise

# Processing parameters
START_DATE = '2023-11-01'
END_DATE = '2025-11-07'
TEMPORAL_RESOLUTION_DAYS = 12  # 12-day composites
TARGET_CRS = "EPSG:32749"  # UTM Zone 49S
TARGET_RESOLUTION = 10  # meters

# Output directory
OUTPUT_DIR = Path('mpc_fusets_data')
OUTPUT_DIR.mkdir(exist_ok=True)

print(f"\n{'='*70}")
print("📋 PROCESSING CONFIGURATION")
print(f"{'='*70}")
print(f"   Study area: Klambu-Glapan paddy fields")
print(f"   Bounding box: {bbox}")
print(f"   Date range: {START_DATE} to {END_DATE}")
print(f"   Temporal resolution: {TEMPORAL_RESOLUTION_DAYS}-day composites")
print(f"   Spatial resolution: {TARGET_RESOLUTION}m")
print(f"   Target CRS: {TARGET_CRS}")
print(f"   Output directory: {OUTPUT_DIR}/")
print(f"{'='*70}")
</VSCode.Cell>
<VSCode.Cell language="markdown">
## 3. Generate 12-Day Composite Periods
</VSCode.Cell>
<VSCode.Cell language="python">
def generate_periods(start_date_str, end_date_str, days_per_period=12):
    """
    Generate periods for composite creation
    """
    start_date = datetime.strptime(start_date_str, '%Y-%m-%d')
    end_date = datetime.strptime(end_date_str, '%Y-%m-%d')
    
    periods = []
    period_num = 1
    current_start = start_date
    
    while current_start <= end_date:
        period_end = current_start + timedelta(days=days_per_period - 1)
        
        if period_end > end_date:
            period_end = end_date
        
        periods.append({
            'period': period_num,
            'start_date': current_start,
            'end_date': period_end,
            'start_str': current_start.strftime('%Y-%m-%d'),
            'end_str': period_end.strftime('%Y-%m-%d'),
            'center_date': current_start + timedelta(days=days_per_period // 2),
            'year': current_start.year,
            'month': current_start.month
        })
        
        if period_end >= end_date:
            break
        
        current_start = period_end + timedelta(days=1)
        period_num += 1
    
    return periods

# Generate periods
periods = generate_periods(START_DATE, END_DATE, TEMPORAL_RESOLUTION_DAYS)

print(f"Generated {len(periods)} periods from {START_DATE} to {END_DATE}:")
print(f"\nFirst 5 periods:")
for p in periods[:5]:
    print(f"  Period {p['period']:2d}: {p['start_str']} to {p['end_str']}")

print(f"\nLast 5 periods:")
for p in periods[-5:]:
    print(f"  Period {p['period']:2d}: {p['start_str']} to {p['end_str']}")

print(f"\nCoverage by year:")
for year in [2023, 2024, 2025]:
    count = len([p for p in periods if p['year'] == year])
    print(f"  {year}: {count} periods")

# Create DataFrame
periods_df = pd.DataFrame(periods)
print(f"\nTotal: {len(periods)} periods covering ~2 years")
</VSCode.Cell>
<VSCode.Cell language="markdown">
## 4. Connect to Microsoft Planetary Computer
</VSCode.Cell>
<VSCode.Cell language="python">
# ============================================================================
# CONNECT TO MICROSOFT PLANETARY COMPUTER STAC API
# ============================================================================

print("="*70)
print("🌍 CONNECTING TO MICROSOFT PLANETARY COMPUTER")
print("="*70)

# MPC STAC API endpoint
STAC_API_URL = "https://planetarycomputer.microsoft.com/api/stac/v1"

# Open STAC catalog
catalog = pystac_client.Client.open(
    STAC_API_URL,
    modifier=planetary_computer.sign_inplace
)

print(f"\n✅ Connected to Microsoft Planetary Computer!")
print(f"   STAC API: {STAC_API_URL}")
print(f"   Catalog: {catalog.title}")

# List available collections
print(f"\n📚 Available Sentinel collections:")
collections = catalog.get_collections()
for collection in collections:
    if 'sentinel' in collection.id.lower():
        print(f"   • {collection.id}: {collection.title}")

print(f"\n✅ Ready to search for Sentinel-1 and Sentinel-2 data!")
</VSCode.Cell>
<VSCode.Cell language="markdown">
## 5. Search and Download Sentinel-2 Data
</VSCode.Cell>
<VSCode.Cell language="python">
def search_sentinel2_items(bbox, start_date, end_date, max_cloud_cover=80):
    """
    Search for Sentinel-2 L2A items in MPC
    """
    search = catalog.search(
        collections=["sentinel-2-l2a"],
        bbox=bbox,
        datetime=f"{start_date}/{end_date}",
        query={
            "eo:cloud_cover": {"lt": max_cloud_cover}
        }
    )
    
    items = list(search.items())
    print(f"   Found {len(items)} Sentinel-2 scenes")
    
    return items

def load_sentinel2_ndvi(items, bbox, resolution=10):
    """
    Load Sentinel-2 data and calculate NDVI
    """
    if not items:
        return None
    
    ndvi_arrays = []
    
    for item in items:
        try:
            # Get B04 (Red) and B08 (NIR) assets
            red_href = item.assets["B04"].href
            nir_href = item.assets["B08"].href
            
            # Sign URLs
            red_href = planetary_computer.sign(red_href)
            nir_href = planetary_computer.sign(nir_href)
            
            # Load bands
            red = rioxarray.open_rasterio(red_href, masked=True).squeeze()
            nir = rioxarray.open_rasterio(nir_href, masked=True).squeeze()
            
            # Clip to bbox
            red = red.rio.clip_box(*bbox)
            nir = nir.rio.clip_box(*bbox)
            
            # Calculate NDVI
            ndvi = (nir - red) / (nir + red)
            ndvi = ndvi.where(np.isfinite(ndvi))
            
            ndvi_arrays.append(ndvi)
            
        except Exception as e:
            print(f"      Error loading scene: {e}")
            continue
    
    if not ndvi_arrays:
        return None
    
    # Stack and compute median
    stacked = xr.concat(ndvi_arrays, dim='time')
    median_ndvi = stacked.median(dim='time', skipna=True)
    
    return median_ndvi

print("="*70)
print("🛰️  PROCESSING SENTINEL-2 DATA")
print("="*70)

s2_periods_data = []

for i, period in enumerate(periods[:3]):  # Start with first 3 periods for testing
    print(f"\nPeriod {period['period']:2d}: {period['start_str']} to {period['end_str']}")
    
    # Search for items
    items = search_sentinel2_items(
        bbox, 
        period['start_str'], 
        period['end_str'],
        max_cloud_cover=80
    )
    
    if items:
        # Load and process
        ndvi = load_sentinel2_ndvi(items, bbox, resolution=TARGET_RESOLUTION)
        
        if ndvi is not None:
            s2_periods_data.append({
                'period': period['period'],
                'ndvi': ndvi,
                'center_date': period['center_date'],
                'n_scenes': len(items)
            })
            print(f"   ✅ NDVI computed from {len(items)} scenes")
        else:
            print(f"   ⚠️  Failed to compute NDVI")
    else:
        print(f"   ⚠️  No scenes found")

print(f"\n✅ Processed {len(s2_periods_data)} periods successfully")
</VSCode.Cell>
<VSCode.Cell language="markdown">
## 6. Search and Download Sentinel-1 Data
</VSCode.Cell>
<VSCode.Cell language="python">
def search_sentinel1_items(bbox, start_date, end_date):
    """
    Search for Sentinel-1 GRD items in MPC
    """
    search = catalog.search(
        collections=["sentinel-1-grd"],
        bbox=bbox,
        datetime=f"{start_date}/{end_date}",
        query={
            "sat:orbit_state": {"eq": "descending"},
            "sar:instrument_mode": {"eq": "IW"}
        }
    )
    
    items = list(search.items())
    print(f"   Found {len(items)} Sentinel-1 scenes")
    
    return items

def load_sentinel1_bands(items, bbox):
    """
    Load Sentinel-1 VV and VH bands
    """
    if not items:
        return None, None
    
    vv_arrays = []
    vh_arrays = []
    
    for item in items:
        try:
            # Get VV and VH assets
            vv_href = planetary_computer.sign(item.assets["vv"].href)
            vh_href = planetary_computer.sign(item.assets["vh"].href)
            
            # Load bands
            vv = rioxarray.open_rasterio(vv_href, masked=True).squeeze()
            vh = rioxarray.open_rasterio(vh_href, masked=True).squeeze()
            
            # Clip to bbox
            vv = vv.rio.clip_box(*bbox)
            vh = vh.rio.clip_box(*bbox)
            
            vv_arrays.append(vv)
            vh_arrays.append(vh)
            
        except Exception as e:
            print(f"      Error loading scene: {e}")
            continue
    
    if not vv_arrays:
        return None, None
    
    # Stack and compute median
    vv_stacked = xr.concat(vv_arrays, dim='time')
    vh_stacked = xr.concat(vh_arrays, dim='time')
    
    vv_median = vv_stacked.median(dim='time', skipna=True)
    vh_median = vh_stacked.median(dim='time', skipna=True)
    
    return vv_median, vh_median

print("="*70)
print("📡 PROCESSING SENTINEL-1 DATA")
print("="*70)

s1_periods_data = []

for i, period in enumerate(periods[:3]):  # Start with first 3 periods for testing
    print(f"\nPeriod {period['period']:2d}: {period['start_str']} to {period['end_str']}")
    
    # Search for items
    items = search_sentinel1_items(bbox, period['start_str'], period['end_str'])
    
    if items:
        # Load and process
        vv, vh = load_sentinel1_bands(items, bbox)
        
        if vv is not None and vh is not None:
            s1_periods_data.append({
                'period': period['period'],
                'vv': vv,
                'vh': vh,
                'center_date': period['center_date'],
                'n_scenes': len(items)
            })
            print(f"   ✅ VV/VH computed from {len(items)} scenes")
        else:
            print(f"   ⚠️  Failed to load data")
    else:
        print(f"   ⚠️  No scenes found")

print(f"\n✅ Processed {len(s1_periods_data)} periods successfully")
</VSCode.Cell>
<VSCode.Cell language="markdown">
## 7. Combine S1 and S2 Data
</VSCode.Cell>
<VSCode.Cell language="python">
print("="*70)
print("🔗 COMBINING SENTINEL-1 AND SENTINEL-2 DATA")
print("="*70)

# Match periods and combine
combined_data = []

for s2_data in s2_periods_data:
    period = s2_data['period']
    
    # Find matching S1 data
    s1_data = next((s for s in s1_periods_data if s['period'] == period), None)
    
    if s1_data is not None:
        print(f"\nPeriod {period}: Combining S1 and S2...")
        
        # Get S2 NDVI as reference
        ndvi = s2_data['ndvi']
        
        # Reproject S1 to match S2
        vv = s1_data['vv'].rio.reproject_match(ndvi)
        vh = s1_data['vh'].rio.reproject_match(ndvi)
        
        # Create combined dataset
        combined = xr.Dataset({
            'VV': vv,
            'VH': vh,
            'S2ndvi': ndvi
        })
        
        # Add time coordinate
        combined = combined.assign_coords(
            t=s2_data['center_date']
        )
        
        combined_data.append(combined)
        
        print(f"   ✅ Combined dataset created")
        print(f"      Shape: {ndvi.shape}")
        print(f"      CRS: {ndvi.rio.crs}")

print(f"\n✅ Combined {len(combined_data)} periods")
</VSCode.Cell>
<VSCode.Cell language="markdown">
## 8. Create Time Series Dataset
</VSCode.Cell>
<VSCode.Cell language="python">
if combined_data:
    print("="*70)
    print("📊 CREATING TIME SERIES DATASET")
    print("="*70)
    
    # Concatenate along time dimension
    timeseries = xr.concat(combined_data, dim='t')
    
    # Ensure proper dimension order (t, y, x)
    timeseries = timeseries.transpose('t', 'y', 'x')
    
    # Add metadata
    timeseries.attrs.update({
        'title': 'Sentinel-1/2 Time Series from Microsoft Planetary Computer',
        'description': '12-day composite periods for FuseTS processing',
        'study_area': 'Klambu-Glapan paddy fields, Demak, Central Java',
        'date_range': f'{START_DATE} to {END_DATE}',
        'temporal_resolution': f'{TEMPORAL_RESOLUTION_DAYS}-day composites',
        'spatial_resolution': f'{TARGET_RESOLUTION}m',
        'crs': str(timeseries.rio.crs),
        'source': 'Microsoft Planetary Computer',
        'collections': 'sentinel-1-grd, sentinel-2-l2a'
    })
    
    print(f"\n✅ Time series dataset created!")
    print(f"\n{timeseries}")
    print(f"\nDimensions: {dict(timeseries.dims)}")
    print(f"Variables: {list(timeseries.data_vars)}")
    print(f"Time steps: {len(timeseries.t)}")
    print(f"CRS: {timeseries.rio.crs}")
    
    # Save to NetCDF
    output_file = OUTPUT_DIR / f'mpc_s1_s2_timeseries_{START_DATE}_{END_DATE}.nc'
    timeseries.to_netcdf(output_file)
    
    print(f"\n💾 Saved to: {output_file}")
    
    # Quick visualization
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    # Mean VV
    timeseries['VV'].mean(dim='t').plot(ax=axes[0], cmap='gray')
    axes[0].set_title('Mean VV (dB)', fontweight='bold')
    
    # Mean VH
    timeseries['VH'].mean(dim='t').plot(ax=axes[1], cmap='gray')
    axes[1].set_title('Mean VH (dB)', fontweight='bold')
    
    # Mean NDVI
    timeseries['S2ndvi'].mean(dim='t').plot(ax=axes[2], cmap='RdYlGn', vmin=-0.2, vmax=0.8)
    axes[2].set_title('Mean NDVI', fontweight='bold')
    
    plt.tight_layout()
    plt.savefig(OUTPUT_DIR / 'mpc_timeseries_preview.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"\n📊 Preview saved: {OUTPUT_DIR / 'mpc_timeseries_preview.png'}")

else:
    print("\n⚠️  No combined data available to create time series")
</VSCode.Cell>
<VSCode.Cell language="markdown">
## 9. Process All Periods (Full Workflow)

Now that we've tested the workflow with 3 periods, let's process all periods.
</VSCode.Cell>
<VSCode.Cell language="python">
# ============================================================================
# FULL PROCESSING FOR ALL PERIODS
# ============================================================================

print("="*70)
print("🚀 STARTING FULL PROCESSING FOR ALL PERIODS")
print("="*70)

def process_single_period(period, bbox, resolution=10, max_cloud_cover=80):
    """
    Process a single period: download S1 and S2, combine, return dataset
    """
    print(f"\n{'─'*70}")
    print(f"Period {period['period']:2d}: {period['start_str']} to {period['end_str']}")
    print(f"{'─'*70}")
    
    result = {
        'period': period['period'],
        'center_date': period['center_date'],
        'dataset': None,
        'n_s1_scenes': 0,
        'n_s2_scenes': 0
    }
    
    # Search Sentinel-2
    print(f"🛰️  Searching Sentinel-2...")
    s2_items = search_sentinel2_items(bbox, period['start_str'], period['end_str'], max_cloud_cover)
    result['n_s2_scenes'] = len(s2_items)
    
    # Search Sentinel-1
    print(f"📡 Searching Sentinel-1...")
    s1_items = search_sentinel1_items(bbox, period['start_str'], period['end_str'])
    result['n_s1_scenes'] = len(s1_items)
    
    if not s2_items and not s1_items:
        print(f"   ⚠️  No data found for this period")
        return result
    
    # Load S2 NDVI
    ndvi = None
    if s2_items:
        print(f"   Loading S2 NDVI...")
        ndvi = load_sentinel2_ndvi(s2_items, bbox, resolution)
    
    # Load S1 VV/VH
    vv, vh = None, None
    if s1_items:
        print(f"   Loading S1 VV/VH...")
        vv, vh = load_sentinel1_bands(s1_items, bbox)
    
    # Combine if we have data
    if ndvi is not None or (vv is not None and vh is not None):
        # Use NDVI as reference if available, otherwise use VV
        reference = ndvi if ndvi is not None else vv
        
        # Create dataset
        ds_dict = {}
        
        if vv is not None and vh is not None:
            ds_dict['VV'] = vv.rio.reproject_match(reference) if vv is not None else reference * 0
            ds_dict['VH'] = vh.rio.reproject_match(reference) if vh is not None else reference * 0
        
        if ndvi is not None:
            ds_dict['S2ndvi'] = ndvi
        
        ds = xr.Dataset(ds_dict)
        ds = ds.assign_coords(t=period['center_date'])
        
        result['dataset'] = ds
        
        print(f"   ✅ Period processed successfully")
        print(f"      S1 scenes: {result['n_s1_scenes']}, S2 scenes: {result['n_s2_scenes']}")
    else:
        print(f"   ⚠️  Failed to create dataset")
    
    return result

# Process all periods
print(f"\n⏳ Processing {len(periods)} periods...")
print(f"   This may take 30-60 minutes depending on your connection\n")

all_results = []

for period in periods:
    result = process_single_period(
        period, 
        bbox, 
        resolution=TARGET_RESOLUTION,
        max_cloud_cover=80
    )
    all_results.append(result)

# Filter successful results
successful_results = [r for r in all_results if r['dataset'] is not None]

print(f"\n{'='*70}")
print(f"✅ PROCESSING COMPLETE!")
print(f"{'='*70}")
print(f"   Total periods: {len(periods)}")
print(f"   Successful: {len(successful_results)}")
print(f"   Failed: {len(periods) - len(successful_results)}")

if successful_results:
    # Create final time series
    print(f"\n📊 Creating final time series dataset...")
    
    final_timeseries = xr.concat(
        [r['dataset'] for r in successful_results],
        dim='t'
    )
    
    # Ensure proper dimension order
    final_timeseries = final_timeseries.transpose('t', 'y', 'x')
    
    # Add comprehensive metadata
    final_timeseries.attrs.update({
        'title': 'Sentinel-1/2 Time Series from Microsoft Planetary Computer',
        'description': f'{TEMPORAL_RESOLUTION_DAYS}-day composite periods for FuseTS MOGPR processing',
        'study_area': 'Klambu-Glapan paddy fields, Demak, Central Java, Indonesia',
        'shapefile_source': shapefile_path,
        'buffer_distance_m': BUFFER_DISTANCE_M,
        'date_range': f'{START_DATE} to {END_DATE}',
        'temporal_resolution': f'{TEMPORAL_RESOLUTION_DAYS}-day composites',
        'spatial_resolution': f'{TARGET_RESOLUTION}m',
        'target_crs': TARGET_CRS,
        'bbox': bbox,
        'n_periods': len(successful_results),
        'source': 'Microsoft Planetary Computer',
        'collections': 'sentinel-1-grd, sentinel-2-l2a',
        'processing_date': datetime.now().isoformat()
    })
    
    print(f"\n{final_timeseries}")
    
    # Save final dataset
    final_output = OUTPUT_DIR / f'mpc_s1_s2_klambu_glapan_{START_DATE}_{END_DATE}_final.nc'
    final_timeseries.to_netcdf(final_output)
    
    print(f"\n💾 Final dataset saved to: {final_output}")
    print(f"   Size: {final_output.stat().st_size / 1e6:.1f} MB")
    
    # Create summary statistics
    summary = pd.DataFrame([
        {
            'period': r['period'],
            'center_date': r['center_date'].strftime('%Y-%m-%d'),
            'n_s1_scenes': r['n_s1_scenes'],
            'n_s2_scenes': r['n_s2_scenes']
        }
        for r in successful_results
    ])
    
    summary_file = OUTPUT_DIR / 'processing_summary.csv'
    summary.to_csv(summary_file, index=False)
    
    print(f"\n📋 Summary saved to: {summary_file}")
    print(f"\n{summary}")
    
else:
    print(f"\n⚠️  No successful results to create time series")
</VSCode.Cell>
<VSCode.Cell language="markdown">
## 10. Visualization and Quality Check
</VSCode.Cell>
<VSCode.Cell language="python">
if 'final_timeseries' in locals() and final_timeseries is not None:
    print("="*70)
    print("📊 CREATING VISUALIZATIONS")
    print("="*70)
    
    # Create comprehensive visualization
    fig = plt.figure(figsize=(20, 12))
    gs = fig.add_gridspec(3, 4, hspace=0.3, wspace=0.3)
    
    # Row 1: Mean values
    ax1 = fig.add_subplot(gs[0, 0])
    final_timeseries['VV'].mean(dim='t').plot(ax=ax1, cmap='gray')
    ax1.set_title('Mean VV (S1)', fontweight='bold')
    ax1.axis('off')
    
    ax2 = fig.add_subplot(gs[0, 1])
    final_timeseries['VH'].mean(dim='t').plot(ax=ax2, cmap='gray')
    ax2.set_title('Mean VH (S1)', fontweight='bold')
    ax2.axis('off')
    
    ax3 = fig.add_subplot(gs[0, 2])
    final_timeseries['S2ndvi'].mean(dim='t').plot(ax=ax3, cmap='RdYlGn', vmin=-0.2, vmax=0.8)
    ax3.set_title('Mean NDVI (S2)', fontweight='bold')
    ax3.axis('off')
    
    ax4 = fig.add_subplot(gs[0, 3])
    final_timeseries['S2ndvi'].std(dim='t').plot(ax=ax4, cmap='viridis')
    ax4.set_title('NDVI Std Dev', fontweight='bold')
    ax4.axis('off')
    
    # Row 2: Data coverage
    ax5 = fig.add_subplot(gs[1, :2])
    vv_coverage = (~np.isnan(final_timeseries['VV'])).sum(dim=['y', 'x'])
    ax5.plot(range(len(vv_coverage)), vv_coverage.values, 'o-', label='VV', linewidth=2)
    vh_coverage = (~np.isnan(final_timeseries['VH'])).sum(dim=['y', 'x'])
    ax5.plot(range(len(vh_coverage)), vh_coverage.values, 's-', label='VH', linewidth=2)
    ax5.set_xlabel('Period')
    ax5.set_ylabel('Valid Pixels')
    ax5.set_title('S1 Data Coverage', fontweight='bold')
    ax5.legend()
    ax5.grid(True, alpha=0.3)
    
    ax6 = fig.add_subplot(gs[1, 2:])
    ndvi_coverage = (~np.isnan(final_timeseries['S2ndvi'])).sum(dim=['y', 'x'])
    ax6.plot(range(len(ndvi_coverage)), ndvi_coverage.values, '^-', color='green', linewidth=2)
    ax6.set_xlabel('Period')
    ax6.set_ylabel('Valid Pixels')
    ax6.set_title('S2 NDVI Coverage', fontweight='bold')
    ax6.grid(True, alpha=0.3)
    
    # Row 3: Sample time series
    ax7 = fig.add_subplot(gs[2, :])
    
    # Extract time series at center pixel
    ny, nx = final_timeseries.dims['y'], final_timeseries.dims['x']
    center_y, center_x = ny // 2, nx // 2
    
    vv_ts = final_timeseries['VV'].isel(y=center_y, x=center_x).values
    vh_ts = final_timeseries['VH'].isel(y=center_y, x=center_x).values
    ndvi_ts = final_timeseries['S2ndvi'].isel(y=center_y, x=center_x).values
    
    ax7_twin = ax7.twinx()
    
    ax7.plot(range(len(vv_ts)), vv_ts, 'o-', label='VV', color='blue', linewidth=2)
    ax7.plot(range(len(vh_ts)), vh_ts, 's-', label='VH', color='cyan', linewidth=2)
    ax7_twin.plot(range(len(ndvi_ts)), ndvi_ts, '^-', label='NDVI', color='green', linewidth=2)
    
    ax7.set_xlabel('Period', fontsize=12)
    ax7.set_ylabel('S1 Backscatter (dB)', fontsize=12)
    ax7_twin.set_ylabel('NDVI', fontsize=12, color='green')
    ax7.set_title(f'Sample Time Series (Center Pixel: y={center_y}, x={center_x})', fontweight='bold')
    ax7.legend(loc='upper left')
    ax7_twin.legend(loc='upper right')
    ax7.grid(True, alpha=0.3)
    
    plt.suptitle(f'Klambu-Glapan S1/S2 Time Series Analysis\n{START_DATE} to {END_DATE} ({len(successful_results)} periods)',
                 fontsize=16, fontweight='bold')
    
    plt.savefig(OUTPUT_DIR / 'final_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"\n✅ Visualization saved: {OUTPUT_DIR / 'final_analysis.png'}")
    
    # Print data quality summary
    print(f"\n{'='*70}")
    print("📊 DATA QUALITY SUMMARY")
    print(f"{'='*70}")
    
    for var in ['VV', 'VH', 'S2ndvi']:
        total = final_timeseries[var].size
        valid = np.isfinite(final_timeseries[var]).sum().values
        coverage = 100 * valid / total
        print(f"   {var:8s}: {valid:,} / {total:,} pixels ({coverage:.1f}% coverage)")
    
    print(f"\n✅ Data preparation complete!")
    print(f"\n📁 Output files in: {OUTPUT_DIR}/")
    print(f"   • mpc_s1_s2_klambu_glapan_{START_DATE}_{END_DATE}_final.nc")
    print(f"   • processing_summary.csv")
    print(f"   • final_analysis.png")
    print(f"   • mpc_study_area.png")

else:
    print("\n⚠️  No final time series available for visualization")
</VSCode.Cell>
<VSCode.Cell language="markdown">
## 11. Prepare for FuseTS MOGPR Processing

The data is now ready to use with FuseTS! Here's how to load and process it:
</VSCode.Cell>
<VSCode.Cell language="python">
# Example: Load the data in a FuseTS workflow

print("="*70)
print("📖 FUSETS USAGE EXAMPLE")
print("="*70)

example_code = """
# ============================================================================
# EXAMPLE: Using MPC data with FuseTS MOGPR
# ============================================================================

import xarray as xr
from fusets.mogpr import MOGPRTransformer
from fusets.analytics import phenology

# 1. Load the MPC time series data
data_file = 'mpc_fusets_data/mpc_s1_s2_klambu_glapan_2023-11-01_2025-11-07_final.nc'
fused_data = xr.open_dataset(data_file)

print("Loaded MPC data:")
print(fused_data)

# 2. Apply MOGPR fusion (optional - data is already gap-filled)
mogpr = MOGPRTransformer()
fused_result = mogpr.fit_transform(fused_data)

# 3. Extract phenology metrics for Indonesian agricultural seasons
phenology_metrics = phenology(fused_result['S2ndvi'])

# Access results
sos_times = phenology_metrics.da_sos_times  # Start of Season
pos_times = phenology_metrics.da_pos_times  # Peak of Season
eos_times = phenology_metrics.da_eos_times  # End of Season

print("\\nPhenology extraction complete!")
print(f"Detected seasons across {len(fused_data.t)} periods")

# 4. Multi-season detection (as you did in previous notebook)
# ... (use your existing multi-season detection code)
"""

print(example_code)

# Save example script
example_file = OUTPUT_DIR / 'fusets_usage_example.py'
with open(example_file, 'w') as f:
    f.write(example_code)

print(f"\n💾 Example script saved to: {example_file}")

print(f"\n{'='*70}")
print("✅ ALL PROCESSING COMPLETE!")
print(f"{'='*70}")
print(f"\nYour data is ready for FuseTS MOGPR processing!")
print(f"Key advantages of MPC data:")
print(f"  • No GEE authentication needed")
print(f"  • Direct Python integration")
print(f"  • Cloud-optimized GeoTIFFs")
print(f"  • Analysis-ready data (ARD)")
print(f"\nNext steps:")
print(f"  1. Load the .nc file in your MOGPR notebook")
print(f"  2. Apply gap-filling or smoothing if needed")
print(f"  3. Extract phenology metrics")
print(f"  4. Detect multi-season patterns")
print(f"\nHappy analyzing! 🌾📊")
</VSCode.Cell>
````